# Harmonic potential

In [ ]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as mplot3d
import numpy as np
from scipy import integrate

## Simple harmonic oscillators

The potential energy of a harmonic oscillator is $q^2/2$. The force is then $F = -q$, and the Hamiltonian is $H(q, p) = T + U = q^2/2 + p^2/2$.

In [ ]:
def hamiltonian(q, p):
    return np.dot(q, q)/2 + np.dot(p, p)/2

In [ ]:
qq = np.linspace(-1, 1, 10)
pp = np.linspace(-1, 1, 10)

Q, P = np.meshgrid(qq, pp)

E = [[None for _ in range(10)] for _ in range(10)]

for i, p in enumerate(pp):
    for j, q in enumerate(qq):
        E[j][i] = hamiltonian(q, p)
        pass
    pass

In [ ]:
plt.contourf(Q, P, E, 100)
plt.title('Contour of Hamiltonian')
plt.show()

### Equations of motion

The equations of motion are $\dot{q} = p$ and $\dot{p} = -q$.

### RK4 integration

In [ ]:
# RK4 approximation.
q_init = np.array([1, 0])
p_init = np.array([-1, 2])

# time series.
tt = np.linspace(0, 100, 1000)

# time data.
n = len(tt)
t_min, t_max = tt[0], tt[n-1]
h = (t_max - t_min) / (n - 1)

# solve for solution.
qq = [None for _ in range(n)]
pp = [None for _ in range(n)]

# compute energies.
energy = [None for _ in range(n)]

for i in range(n):
    if i == 0:
        qq[0] = q_init
        pp[0] = p_init
        energy[0] = hamiltonian(q_init, p_init)
        pass
    else:
        q, p = qq[i-1], pp[i-1]
        
        # compute k1.
        k1q = h*p
        k1p = -h*q
        
        # compute k2.
        k2q = h*(p + k1q/2)
        k2p = -h*(q + k1p/2)
        
        # compute k3.
        k3q = h*(p + k2q/2)
        k3p = -h*(q + k2p/2)
        
        # compute k4.
        k4q = h*(p + k3q)
        k4p = - h*(q + k3p)
        
        # iterate.
        qq[i], pp[i] = q + 1/6*(k1q + 2*k2q + 2*k3q + k4q), p + 1/6*(k1p + 2*k2p + 2*k3p + k4p)
        
        # compute energy.
        energy[i] = hamiltonian(q, p)
        pass
    pass

In [ ]:
xx = [q[0] for q in qq]
yy = [q[1] for q in qq]

f, axs = plt.subplots(3, 1, figsize=(6.4 * 1, 4.8 * 3))

plt.subplot(3, 1, 1)
plt.plot(tt, xx)
plt.plot(tt, yy)
plt.title('Position evolution graph')

plt.subplot(3, 1, 2)
plt.plot(xx, yy)
plt.title('Solution space graph')

plt.subplot(3, 1, 3)
plt.plot(tt, energy)
plt.title('Energy evolution graph')
plt.show()

In the RK4 case, energy grows exponentially, and mechanical energy is not conserved.

In [ ]:
print('Standard deviation: {}'.format(np.std(energy)))

### Verlet integration

In [ ]:
# solve.
qq = [None for _ in range(n)]
pp = [None for _ in range(n)]
energy = [None for _ in range(n)]

for i in range(n):
    if i == 0:
        qq[0] = q_init
        pp[0] = p_init
        energy[0] = hamiltonian(q_init, p_init)
        pass
    else:
        q, p = qq[i-1], pp[i-1]
        
        _p = p - h*q/2
        q_new = q + h*_p
        acceleration = -q_new
        p_new = _p + h*acceleration/2
        
        qq[i], pp[i] = q_new, p_new
        energy[i] = hamiltonian (q_new, p_new)
        pass
    pass

In [ ]:
xx = [q[0] for q in qq]
yy = [q[1] for q in qq]

f, axs = plt.subplots(3, 1, figsize=(6.4 * 1, 4.8 * 3))

plt.subplot(3, 1, 1)
plt.plot(tt, xx)
plt.plot(tt, yy)
plt.title('Position evolution graph')

plt.subplot(3, 1, 2)
plt.plot(xx, yy)
plt.title('Solution space graph')

plt.subplot(3, 1, 3)
plt.plot(tt, energy)
plt.title('Energy evolution graph')
plt.show()

In [ ]:
print('Standard deviation: {}'.format(np.std(energy)))

In the Verlet case, energy is bounded between $0.498$ and $0.5$. Verlet integration conserves the system's mechanical energy.

## Stochastic Differential Equations

The function `sample_unit_sphere` samples `n` random unit vectors, each vector of dimension `dim`.

In [ ]:
def norm(v):
    return np.sqrt(v.dot(v))

def sample_unit_sphere(states, dimension):
    if states == 1:
        v = np.array([np.random.normal(0, 1) for _ in range(dimension)])
        return v/norm(v)
    else:
        return np.array([sample_unit_sphere(1, dimension) for _ in range(states)])
    pass

In [ ]:
sample_unit_sphere(5, 2)

The normal distribution, $\dfrac{1}{\sqrt{2\pi\sigma^2}}\exp\bigg(-\dfrac{(x-\mu)^2}{2\sigma^2}\bigg)$.

In [ ]:
def normal_distribution(x, mu = 0, sigma = 1):
    return 1/np.sqrt(2*np.pi*sigma)*np.exp(-(x-mu)**2/(2*sigma**2))

In [ ]:
xx = np.linspace(-5, 5, 1000)
plt.plot(xx, normal_distribution(xx))
plt.title('Normal distribution')
plt.show()

### Wiener processes

A Wiener process is a stochastic differential equation, where $W(t)$ is a random variable which by the Central Limit Theorem satisfies $W(t)\sim\mathcal{N}(0, t)$.


In [ ]:
def wiener_differential(states, dimension, tt):
    # states: number of particles in the initial state.
    n = len(tt)
    t_min, t_max = tt[0], tt[n-1]
    dt = (t_max - t_min) / (n - 1)
    
    for i in range(n):
        yield np.sqrt(dt)*sample_unit_sphere(states, dimension)

In [ ]:
def wiener_process(initial_state, tt):
    states = len(initial_state)
    dimension = len(initial_state[0])
    
    current_state = initial_state
    
    for i, dw in enumerate(wiener_differential(states, dimension, tt)):
        if i == 0:
            yield current_state
            pass
        
        else:
            current_state = current_state + dw
            yield current_state
            pass
        pass
    pass

In [ ]:
initial_state = np.array([[0] for _ in range(10**3)])
tt = np.linspace(0, 1, 10**3)

solution = [w.flatten() for w in wiener_process(initial_state, tt)]

In [ ]:
f, axs = plt.subplots(2, 1, figsize = (6.4*1, 4.8*2))

plt.subplot(2, 1, 1)
plt.plot(tt, solution)

plt.subplot(2, 1, 2)
plt.hist(solution[-1], 100, density = True)

x_min, x_max = plt.xlim()
xx = np.linspace(x_min, x_max, 1000)
plt.plot(xx, normal_distribution(xx), label = 'normal distribution fit')
plt.legend()

plt.show()

### Ornstein Uhlenbeck process

The Ornstein Uhlenbeck (OU) process $$dp = -\gamma pdt + \sqrt{2\gamma kTm}dW,$$ samples in the long term, the probability density $$\rho(u) = \dfrac{1}{\sqrt{2\pi mkT}}\exp\bigg(-\dfrac{1}{kT}\dfrac{u^2}{2m}\bigg).$$

In [ ]:
def gibbs_boltzmann_distribution(u, gamma = 1, k = 1, T = 1, m = 1):
    return 1/np.sqrt(2*np.pi*m*k*T) * np.exp(-1/(k*T)*u**2/(2*m))

In [ ]:
xx = np.linspace(-10, 10, 10000)
plt.plot(xx, gibbs_boltzmann_distribution(xx))
plt.title('Gibbs Boltzmann distribution')
plt.show()

In [ ]:
def ou_euler_maruyama(initial_state, tt, gamma = 1, k = 1, T = 1, m = 1):
    states = len(initial_state)
    dimension = len(initial_state[0])
    
    n = len(tt)
    t_min, t_max = tt[0], tt[n-1]
    dt = (t_max - t_min) / (n - 1)
    
    current_state = initial_state
    for i, dw in enumerate(wiener_differential(states, dimension, tt)):
        if i == 0:
            yield current_state
            pass
        else:
            current_state = current_state - gamma*current_state*dt + np.sqrt(2*gamma*k*T*m)*dw
            yield current_state
            pass
        pass
    pass

In [ ]:
initial_state = np.array([[20] for _ in range(10**3)])
tt = np.linspace(0, 10, 1000)

solution = [w.flatten() for w in ou_euler_maruyama(initial_state, tt)]

In [ ]:
f, axs = plt.subplots(2, 1, figsize = (6.4*1, 4.8*2))

plt.subplot(2, 1, 1)
plt.plot(tt, solution)

plt.subplot(2, 1, 2)
plt.hist(solution[-1], 100, density = True)

x_min, x_max = plt.xlim()
xx = np.linspace(x_min, x_max, 10000)
plt.plot(xx, gibbs_boltzmann_distribution(xx), label = 'Gibbs Boltzmann distribution fit')
plt.legend()

plt.show()

In the limit of high $\gamma$, the Gibbs-Boltzmann distribution no longer applies.

### Langevin dynamics

The equations for Langevin dynamics are $$\begin{split}
dq &= M^{-1}pdt,\\
dp &= - qdt - \gamma pdt + \sqrt{2\gamma kT}M^{1/2}dW,
\end{split}$$ where the gradient $\nabla U(q)$ reduces to $q$.

In [ ]:
def langevin_dynamics_iterator(q, p, dt, dw, m = 1, gamma = 1, k = 1, T = 1):
    dq = dt/m * p
    dp = - dt*q - dt*gamma*p + dw*np.sqrt(2*gamma*k*T*m)
    return [q + dq, p + dp]

In [ ]:
def langevin_dynamics(initial_state, tt, gamma = 1, k = 1, T = 1, m = 1):
    q_init, p_init = initial_state
    states = len(q_init)
    dimension = len(q_init[0])
    
    n = len(tt)
    t_min, t_max = tt[0], tt[n-1]
    dt = (t_max - t_min) / (n - 1)
    
    current_state = initial_state
    for i, dw in enumerate(wiener_differential(states, dimension, tt)):
        if i == 0:
            yield current_state
            pass
        else:
            current_q, current_p = current_state
            new_q, new_p = langevin_dynamics_iterator(current_q, current_p, dt, dw)
            current_state = [new_q, new_p]
            yield current_state
            pass
        pass
    pass

In [ ]:
initial_state = [
    np.array([[0] for _ in range(10**3)]), # initial position
    np.array([[0] for _ in range(10**3)])  # initial momentum
]
tt = np.linspace(0, 1, 100)
solution = [w for w in langevin_dynamics(initial_state, tt)]

# extract positions.
qq = [s[0].flatten() for s in solution]

In [ ]:
f, axs = plt.subplots(2, 1, figsize = (6.4*1, 4.8*2))

plt.subplot(2, 1, 1)
plt.plot(tt, qq)

plt.subplot(2, 1, 2)
plt.hist(qq[-1], 100, density = True)

plt.show()